<a href="https://colab.research.google.com/github/whispermy/novatus_academia/blob/main/pbl/pbl_lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/pbl/trainingData2.csv")
data.head()

,INPUT,OUTPUT,종류
0,A는 B,A = B,선언
1,A는 B로 설정,A = B,선언
2,A는 B로 초기화,A = B,선언
3,A를 B로,A = B,선언
4,A에다가 입력받아줘,A = input(),입력


In [ ]:
#형태소 분석기 사용하기 
!pip install konlpy

     |████████████████████████████████| 19.4 MB 49.9 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 448 kB 40.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Kkma, Komoran, Okt, Mecab

okt = Okt()
txt = data['INPUT'][199]
print(okt.pos(txt,norm=True, stem=False, join=False))

[('만약', 'Noun'), ('A', 'Alpha'), ('가', 'Verb'), ('NonE', 'Alpha'), ('값', 'Noun'), ('이', 'Josa'), ('아닐', 'Adjective'), ('때', 'Noun')]


In [ ]:
list_input = []

for txt in data['INPUT']:
  tokens = okt.pos(txt,norm=True, stem=False, join=False)
  s = []
  for token in tokens:
    s.append(token[0])
  list_input.append(s)

list_input[:10]


[['A', '는', 'B'],
 ['A', '는', 'B', '로', '설정'],
 ['A', '는', 'B', '로', '초기', '화'],
 ['A', '를', 'B', '로'],
 ['A', '에다가', '입력', '받아줘'],
 ['A', '에', '입력', '받아'],
 ['A', '에', '문자열', 'B', '로', '초기', '화'],
 ['A', '를', '문자열', 'B', '로'],
 ['문자열', 'A', '를', '출력', '해'],
 ['A', '를', '출력', '해']]

In [ ]:
#word2vec
from gensim.models import Word2Vec
dimension_word = 50
# train word2vec model
model_input = Word2Vec(list_input, size=dimension_word, window=2, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model_input.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 150


In [ ]:
input_vector = model_input.wv
input_vector['문자열']

array([-6.76404126e-03,  8.46563280e-03,  6.07652077e-03, -5.25879487e-03,
       -3.45286122e-03,  1.98760885e-03,  7.48767098e-03, -9.31620225e-03,
        1.00731021e-02,  4.14068904e-03,  9.40385181e-03, -5.15341107e-03,
       -7.05679599e-03, -1.11462297e-02,  1.71907435e-04,  8.07250105e-03,
        2.69089430e-03,  5.37000051e-05, -8.17278051e-04,  6.61294069e-03,
       -1.05456915e-02,  5.45349531e-03,  9.33601055e-03, -1.21585536e-03,
        7.28258118e-03,  8.58709309e-03,  4.90114884e-03, -4.92287800e-03,
        9.67736938e-04, -2.47530593e-03, -4.02876828e-03, -4.40766755e-03,
       -8.26888531e-03, -5.33263013e-03,  3.89395375e-03, -5.85786859e-03,
       -6.00622781e-03,  7.46226450e-03,  1.60263677e-03,  8.06033029e-04,
        8.97885184e-04,  3.35023063e-03, -4.89797676e-03, -8.53381027e-03,
       -4.25724871e-03,  5.49170747e-03, -7.88474747e-04,  3.17527330e-03,
       -1.54238939e-03, -3.55152087e-03], dtype=float32)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer_input = Tokenizer(lower=False)
tokenizer_input.fit_on_texts(list_input)
encoded_input = tokenizer_input.texts_to_sequences(list_input)
encoded_input[:10]

[[1, 41, 2],
 [1, 41, 2, 15, 83],
 [1, 41, 2, 15, 27, 52],
 [1, 10, 2, 15],
 [1, 116, 84, 117],
 [1, 53, 84, 118],
 [1, 53, 61, 2, 15, 27, 52],
 [1, 10, 61, 2, 15],
 [61, 1, 10, 47, 22],
 [1, 10, 47, 22]]

In [ ]:
#input 가중치 행렬
import numpy as np

input_vector_dic = {}
for key in input_vector.wv.vocab.keys():
    input_vector_dic[key] = input_vector[key]
input_vector_dic['A']

# total vocabulary size plus 0 for unknown words
vocab_size=len(tokenizer_input.word_index)+1
input_embedding_vectors = np.zeros((vocab_size, dimension_word))
for word, i in tokenizer_input.word_index.items():
    input_embedding_vectors[i] = input_vector_dic.get(word)

input_embedding_vectors[np.isnan(input_embedding_vectors)] = 0

#정규화 
input_embedding_vectors = input_embedding_vectors / input_embedding_vectors.max(axis=0)

input_embedding_vectors.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


(151, 50)

In [ ]:
list_output_in = []
list_output_target = []

########################################################
######## decoder 의 input 으로 쓸 데이터 만들기 ########
########################################################
#(decoder의 데이터는 띄어쓰기 단위로 자르기)
for txt in data['OUTPUT']:
  tokens = txt.split()
  s = ['<sos>']
  t = []
  for token in tokens:
    s.append(token)
    t.append(token)
  list_output_in.append(s)
  t.append('<eos>')
  list_output_target.append(t)
  
print(list_output_in[:10])
print(list_output_target[:10])

[['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'input()'], ['<sos>', 'A', '=', 'input()'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'print("', 'A', '")'], ['<sos>', 'print(', 'A', ')']]
[['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'input()', '<eos>'], ['A', '=', 'input()', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['print("', 'A', '")', '<eos>'], ['print(', 'A', ')', '<eos>']]


In [ ]:
# list_output_target = []

########################################################
######## decoder 의 output 으로 쓸 데이터 만들기 ########
########################################################
#(decoder의 데이터는 띄어쓰기 단위로 자르기)

In [ ]:
tokenizer_output = Tokenizer(lower=False)
tokenizer_output.fit_on_texts(list_output_in)
tokenizer_output.fit_on_texts(list_output_target)

encoded_output_in = tokenizer_output.texts_to_sequences(list_output_in)
print(encoded_output_in[:10])

encoded_output_target = tokenizer_output.texts_to_sequences(list_output_target)
print(encoded_output_target[:10])

[[4, 1, 17, 2], [4, 1, 17, 2], [4, 1, 17, 2], [4, 1, 17, 2], [4, 1, 17, 35], [4, 1, 17, 35], [4, 1, 17, 2], [4, 1, 17, 2], [4, 36, 1, 37], [4, 31, 1, 15]]
[[1, 17, 2, 5], [1, 17, 2, 5], [1, 17, 2, 5], [1, 17, 2, 5], [1, 17, 35, 5], [1, 17, 35, 5], [1, 17, 2, 5], [1, 17, 2, 5], [36, 1, 37, 5], [31, 1, 15, 5]]


In [ ]:
#word2vec
from gensim.models import Word2Vec

# train word2vec model
model_output = Word2Vec(list_output_in, size=dimension_word, window=2, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model_output.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 43


In [ ]:
output_vector = model_output.wv
output_vector['<sos>']

array([-1.4058262e-02,  5.5063305e-05,  1.2311778e-03, -3.5225968e-03,
       -1.0308884e-03,  9.4542885e-03, -3.0672147e-03, -7.6089692e-03,
        8.0764908e-03,  9.7000906e-03, -1.1631525e-02, -1.1188308e-02,
       -3.8223807e-03, -1.3168613e-02, -1.2614398e-03, -6.0122106e-03,
       -1.2406388e-02, -2.2140185e-03, -6.3734739e-03, -1.2454268e-02,
        5.2389577e-03, -2.4734356e-03,  1.8475259e-04, -1.1181596e-02,
       -2.1001678e-03, -8.9886095e-03, -1.0086668e-02, -8.5689100e-03,
        9.9031357e-03, -9.1238590e-03,  1.2742590e-02,  5.9764180e-03,
        2.7814996e-03, -3.4800558e-03, -5.7395771e-03, -9.3580838e-03,
        1.5456112e-03, -1.3557404e-02, -3.4456886e-04, -8.8452324e-03,
       -4.7580223e-03, -3.1288785e-03,  1.3976433e-02, -5.0580688e-03,
        1.9398787e-03,  5.8600400e-03, -7.4126460e-03,  1.5291342e-02,
        3.2823028e-03, -1.1630162e-02], dtype=float32)

In [ ]:
output_vector_dic = {}
for key in model_output.wv.vocab.keys():
    output_vector_dic[key] = output_vector[key]

In [ ]:
# total vocabulary size plus 0 for unknown words
vocab_size=len(tokenizer_output.word_index)+1
output_embedding_vectors = np.zeros((vocab_size, dimension_word))
for word, i in tokenizer_output.word_index.items():
    output_embedding_vectors[i] = output_vector_dic.get(word)

output_embedding_vectors[np.isnan(output_embedding_vectors)] = 0

#정규화 
output_embedding_vectors = output_embedding_vectors / output_embedding_vectors.max(axis=0)
output_embedding_vectors.shape

(45, 50)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

encoder_max_length = max([len(s) for s in list_input])
encoder_input = pad_sequences(encoded_input, maxlen=encoder_max_length, padding='post')
print(encoder_input.shape)

decoder_max_length_in = max([len(s) for s in list_output_in])
decoder_input = pad_sequences(encoded_output_in, maxlen=decoder_max_length_in, padding='post')
print(decoder_input.shape)

decoder_max_length_target = max([len(s) for s in list_output_target])
decoder_target = pad_sequences(encoded_output_target, maxlen=decoder_max_length_target, padding='post')
print(decoder_target.shape)

(257, 26)
(257, 14)
(257, 14)


In [ ]:
np.random.seed(50)
np.random.shuffle(encoder_input)
np.random.seed(50)
np.random.shuffle(decoder_input)
np.random.seed(50)
np.random.shuffle(decoder_target)

print(encoder_input[0])
print(decoder_input[0])
print(decoder_target[0])

[ 1  3  2  4 17 65  5  3  9  6 45  7  8 12  0  0  0  0  0  0  0  0  0  0
  0  0]
[ 4  8  1 20  2 10  6 12  9  3  0  0  0  0]
[ 8  1 20  2 10  6 12  9  3  5  0  0  0  0]


In [ ]:
test_ration = int(len(encoder_input) * 0.1)

In [ ]:
encoder_input_train = encoder_input[:-test_ration]
encoder_input_test = encoder_input[-test_ration:]

decoder_input_train = decoder_input[:-test_ration]
decoder_input_test = decoder_input[-test_ration:]

decoder_target_train = decoder_target[:-test_ration]
decoder_target_test = decoder_target[-test_ration:]

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, GRU, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model

#########################################################################
####################encoder 구조를 rnn로 만들어보기######################
#########################################################################

encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(151,dimension_word,weights=[input_embedding_vectors])(encoder_inputs)
encoder_gru = GRU(dimension_word, activation='relu')(enc_emb)
encoder_output = encoder_gru



In [ ]:
from tensorflow.keras.layers import TimeDistributed

##################################################################
####################decoder 구조 만들기###########################
##################################################################
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(45,dimension_word, weights=[output_embedding_vectors])(decoder_inputs)
decoder_gru = GRU(dimension_word,return_sequences=True, return_state=True, activation='relu')
decoder_outputs,_ = decoder_gru(dec_emb, initial_state=encoder_output)
decoder_dense = TimeDistributed(Dense(45, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
##################################################################
######################모형 연결시키기#############################
##################################################################

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
##################################################################
###################### 모형 컴파일하기 ###########################
##################################################################

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 50)     7550        ['input_2[0][0]']                
                                                                                                  
 embedding_6 (Embedding)        (None, None, 50)     2250        ['input_8[0][0]']                
                                                                                              

In [ ]:
##################################################################
###################### 모형 학습시키기 ###########################
##################################################################

model.fit(x=[encoder_input_train,decoder_input_train], y=decoder_target_train, batch_size=10, epochs=50)


Epoch 1/50
24/24 [==============================] - 11s 169ms/step - loss: 3.5882 - acc: 0.4267
Epoch 2/50
24/24 [==============================] - 4s 175ms/step - loss: 2.2474 - acc: 0.5511
Epoch 3/50
24/24 [==============================] - 4s 172ms/step - loss: 1.3913 - acc: 0.6438
Epoch 4/50
24/24 [==============================] - 4s 171ms/step - loss: 1.0498 - acc: 0.7244
Epoch 5/50
24/24 [==============================] - 4s 171ms/step - loss: 0.8493 - acc: 0.7669
Epoch 6/50
24/24 [==============================] - 4s 171ms/step - loss: 0.7110 - acc: 0.7986
Epoch 7/50
24/24 [==============================] - 4s 175ms/step - loss: 0.6148 - acc: 0.8177
Epoch 8/50
24/24 [==============================] - 4s 173ms/step - loss: 0.5329 - acc: 0.8368
Epoch 9/50
24/24 [==============================] - 4s 168ms/step - loss: 0.4761 - acc: 0.8491
Epoch 10/50
24/24 [==============================] - 4s 169ms/step - loss: 0.4247 - acc: 0.8624
Epoch 11/50
24/24 [=============================

In [ ]:
#########################################################################
####################encoder 구조를 gru로 만들어보기######################
#########################################################################


In [ ]:
#########################################################################
####################encoder 구조를 cnn로 만들어보기######################
#########################################################################
